#**Importing the dataset**

In [0]:
#Importing necessary libraries

import pandas as pd   # For the DataFrame 
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
#%matplotlib notebook                      # In jupyter run this line to get an interactive plot

In [0]:
ndf1=pd.read_csv('dataset/anger-ratings-0to1.train.txt',sep='\t',names=['ind','text','emotion','int'])
ndf2=pd.read_csv('datsset/fear-ratings-0to1.train.txt',sep='\t',names=['ind','text','emotion','int'])
ndf3=pd.read_csv('dataset/joy-ratings-0to1.train.txt',sep='\t',names=['ind','text','emotion','int'])
ndf4=pd.read_csv('datset/sadness-ratings-0to1.train.txt',sep='\t',names=['ind','text','emotion','int'])
df1 = ndf1.append(ndf2).append(ndf3).append(ndf4)
#df1.pop('ind')
df1=df1.sample(frac=1)
df1

In [0]:
import numpy as np

emotions_set=np.unique(df1['emotion'])
print('The Various Emotions are: ',emotions_set,'\n\n\n The Description of the Dataset: \n ')
df1.describe()

In [0]:
df=df1.copy()                                     #Creating a copy for a possible future use

!pip install vaderSentiment

In [0]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

#x= sid.polarity_scores(sent)

df1['lex_polarity']=df1['text'].apply(lambda x: sid.polarity_scores(x)['compound'] )
df1

In [0]:
from collections import Counter

freq=Counter(df1['emotion'])
freq=dict(freq)
x= pd.DataFrame(freq,index=freq.keys())
sns.catplot(kind='bar',data=x)

In [0]:
sns.distplot(df1['lex_polarity']);

In [0]:
#sns.pairplot(df1)

In [0]:
sns.catplot(x='emotion',y='lex_polarity',kind='violin',data=df1)
sns.catplot(x='emotion',y='lex_polarity',kind='swarm',data=df1)

In [0]:
labels_ser=pd.Series(df1['emotion'].values, dtype='category')    # Its a categorical series. That is it converts all the values.
                                                  # within that series into numbers
                                                  
cat_labels=labels_ser.cat.codes 
df1['ctgr_label']=cat_labels
df1['ind']=range(len(df1))
#df1

In [0]:
#sns.catplot(x='lex_polarity',y='ctgr_label',data=df1)

In [0]:
sns.jointplot(x='ind', y="lex_polarity",kind='kde', data=df1);

In [0]:
#Preprocessing modules- Removes @tags, #,(,),\n,\\n,emoticons

def rem_rt(string):
  x=string.split()
  c=0
  for i in range(len(x)):
    if x[i-c][0]=='@':
      x.remove(x[i-c])
      c+=1
  y= ' '.join(x)
  emoji_pattern = re.compile("["
        u"\U0001F600-\U0001FFFF"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'', y)

import re
text=df['text'].values
emotions=df['emotion'].values
ntext=[]
for sub in text:
  x=sub.split()

  sub=sub.replace(r'(','')
  sub=sub.replace(r")",'')
  sub=sub.replace(r"\n",'') 
  sub=sub.replace(r"#",'')
  sub=rem_rt(sub)
  ntext.append(re.sub('\n', '', sub)) 

clean_text=ntext
clean_text

In [0]:
df['clean_text']=clean_text
hybrid=df[['clean_text','int']].values
hybrid

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

vect=TfidfVectorizer()

dtm=vect.fit_transform(clean_text)
dtm_array=dtm.toarray()
dtm_column = vect.get_feature_names()
df_dtm=pd.DataFrame(dtm_array,columns=dtm_column)
#df_dtm

In [0]:
vect=TfidfVectorizer()

dtm=vect.fit_transform(clean_text)

#print(dtm)

dtm_array=dtm.toarray()

dtm_column = vect.get_feature_names()

df_dtm=pd.DataFrame(dtm_array,columns=dtm_column)

#df_dtm['0']=df['int'].values

#df_dtm

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(clean_text, test_size=0.2, random_state=42)

y_train, y_test = train_test_split(emotions, test_size=0.2, random_state=42)


In [0]:
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
 
!pip install autoviml
from autoviml.Auto_NLP import Auto_NLP

In [0]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42)


nlp_column=['text','int']
target='emotion'

train_nlp, test_nlp, nlp_transformer, preds = Auto_NLP(nlp_column, train, test, target,    #nlp_transformer- pipeline
                                                     score_type='balanced_accuracy',
                                                     modeltype='Classification',
                                                     top_num_features=200, 
                                                     verbose=0, build_model=True)

nlp_transformer

In [0]:
x=nlp_transformer

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

x=Pipeline(memory=None,
         steps=[('countvectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 #dtype=<class 'numpy.int64'>,
                                 encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=8741, min_df=2,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents='unicode',
                                 token_pattern='\\w{1,}', tokenizer=None,
                                 vocabulary=None)),
                ('multinomialnb',
                 MultinomialNB(alpha=0.9906273994707961, class_prior=None,
                               fit_prior=True))],
         verbose=False)

In [0]:
x

In [0]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

#nlp_transformer.fit(X_train,y_train)
x.fit(X_train,y_train)
y_pred=x.predict(X_test)
#y_pred=text_model.predict(X_test)

print(accuracy_score(y_test,y_pred),'\n\n')

print(confusion_matrix(y_test,y_pred),'\n\n')

print(classification_report(y_test,y_pred))

In [0]:
emo = ['anger','fear' ,'joy' ,'sadness'] 

def pred_emotion(string, ret=None):
  '''
  If you give "ret" as either - model_output, emotion or dict, it'll retun the same (only that).
  But if you don't give it anything, it'll print all the values
  '''
  emo = ['ANGER','FEAR','JOY','SADNESS'] 
  probs=list(nlp_transformer.predict_proba([string])[0])

  if ret==None:
    print('Model output: ',nlp_transformer.predict_proba([string])[0],'\n\n')
  elif ret=='model_output':
    return nlp_transformer.predict_proba([string])[0]
  
  if ret==None:
    print('The Sentence is mostly "{}"'.format(emo[np.argmax(probs)]),'\n\n')
  elif ret=='emotion':
    return emo[np.argmax(probs)]

  emodict={k:'{:3f}%'.format(v*100) for (k,v) in zip(emo,probs)}
  if ret==None:
    print(emodict,'\n')
  elif ret=='dict':
    return emodict


In [0]:
stri="I'll kill him now if I see his face!!"

pred_emotion(stri)

In [0]:
stri="Corona is dreadful"

pred_emotion(stri)#,'emotion')

In [0]:
stri="He's not your true friend"

pred_emotion(stri,'dict')

#**Taking Twitter Data** 
By scraping from tweepy through a search keyword

In [0]:
import tweepy
import pandas as pd
import nltk

#!pip install langdetect
#nltk.download('vader_lexicon')

auth = tweepy.OAuthHandler('rJcIIAjv5ZrL8NKjWqPhcIkKn', 'LKX7W2BbH5XH9eU1cBanOoW5jl31Vcek18Xy4eeajTzUkH8XOj')

auth.set_access_token('2832840558-2arbFHg83wp3IHDZ9Hnz2sUluIAgEKz5EOBaYi6', 'PntXRl3wopfhLL2U3JnmMJp1Wy2XqwlmuzYCZXqmtz5st')

api = tweepy.API(auth)

#user = api.get_user('narendramodi')

In [0]:


topic= 'obama'            #Set the username of the user
cnt= 100               #Set the number of tweets to be taken.

#posts = api.user_timeline(screen_name = user, count = cnt, tweet_mode='extended')  #Gets all the posts of a certain user

posts = api.search( topic, tweet_mode='extended',count=cnt)

tweets=[]
for i in range(len(posts)):
  ran=posts[i]._json['full_text'].split('https')[0]   #To remove hyperlinks
  tweets.append(ran)

!pip install langdetect
from langdetect import detect

eng_tweets=[]
for i in tweets:
  try:
    if detect(i)=='en':
      s=i.split(': ')[1]
      eng_tweets.append(s)
  except:
    pass
  

eng_tweets

clean_tweets=[]
for sub in eng_tweets:
  sub=sub.replace(r'(','')
  sub=sub.replace(r")",'')
  sub=sub.replace(r"\n",'')
  sub=sub.replace(r"#",'')
  sub=rem_rt(sub)
  clean_tweets.append(re.sub('\n', '', sub))

itr=iter(clean_tweets)
clean_tweets

In [0]:
#Streaming and collecting tweets


from tweepy import StreamListener

class MyStreamListener(StreamListener):

    def on_status(self, status):
        
        #return status.text

        if hasattr(status,"extended_tweet"):
          text = status.extended_tweet["full_text"]
        else:
          text = status.text

        print(text)

        with open("out.csv", "a", encoding='utf-8') as f:
          #f.write("%s\n" % (text))
          pass

myStream = tweepy.Stream(auth = auth , listener=MyStreamListener())

#myStream.filter(track=['trump'])

In [0]:
samp=next(itr)

print(samp,'\n\n')

pred_emotion(samp)


In [0]:

import tweepy
import pandas as pd
import nltk
import re

#!pip install langdetect
#nltk.download('vader_lexicon')

auth = tweepy.OAuthHandler('rJcIIAjv5ZrL8NKjWqPhcIkKn', 'LKX7W2BbH5XH9eU1cBanOoW5jl31Vcek18Xy4eeajTzUkH8XOj')

auth.set_access_token('2832840558-2arbFHg83wp3IHDZ9Hnz2sUluIAgEKz5EOBaYi6', 'PntXRl3wopfhLL2U3JnmMJp1Wy2XqwlmuzYCZXqmtz5st')

api = tweepy.API(auth)

#user = api.get_user('narendramodi')

def scrape_tweets(topic):

  topics= topic           #Set the username of the user
  cnt= 50            #Set the number of tweets to be taken.

  #posts = api.user_timeline(screen_name = user, count = cnt, tweet_mode='extended')  #Gets all the posts of a certain user

  posts = api.search( topics, tweet_mode='extended',count=cnt)

  tweets=[]
  for i in range(len(posts)):
    ran=posts[i]._json['full_text'].split('https')[0]   #To remove hyperlinks
    tweets.append(ran)

  from langdetect import detect

  eng_tweets=[]
  for i in tweets:
    try:
      if detect(i)=='en':
        s=i.split(': ')[1]
        eng_tweets.append(s)
    except:
      pass

  eng_tweets

  clean_tweets=[]
  for sub in eng_tweets:
    sub=sub.replace(r'(','')
    sub=sub.replace(r")",'')
    sub=sub.replace(r"\n",'')
    sub=sub.replace(r"#",'')
    sub=rem_rt(sub)
    clean_tweets.append(re.sub('\n', '', sub))

  itr=iter(clean_tweets)

  return clean_tweets



In [0]:
scrape_tweets('trump')

In [0]:
def tweet_analysis(key):

    tweet=scrape_tweets(key)[0]
    emo=pred_emotion(tweet,'dict')
    return emo


In [0]:
emo=tweet_analysis('dread')
emo